In [10]:
pip install qpsolvers

Note: you may need to restart the kernel to use updated packages.


## Example 

Minimize :  $J = \frac{1}{2}x^\intercal A x  - b^\intercal x $
<br>
Subject to the constraints: 
<br>
$C x \leq f$ with 
$$A = \begin{pmatrix} 3 & 1 & 0 \\ 1 & 3 & 1 \\ 0 & 1 & 3 \end{pmatrix}, \quad  b = \begin{pmatrix} 1 \\ -1 \\2 \end{pmatrix},\quad C = \begin{pmatrix} 1 & 2  \\ 2 & 0  \\ -1 & 2 \end{pmatrix}, \quad  f = \begin{pmatrix} 1 \\ 0\\2 \end{pmatrix}$$


We will use the library qp_solvers to solve this system. The adopted format for this function is given by
$$Z = \frac{1}{2}x^\intercal P x  + q^\intercal x  \quad \mbox{Subject to $G x \leq h$}$$


-It requires A = P to be definite positive

-The matrix C = G should have the same dimension as A. To that aim one can add zeros to the matrix.

-Note that q = -b

## Quadratic programming using QPsolvers

In [11]:
from qpsolvers import solve_qp

import numpy as np
import sympy as sp
from matplotlib import pyplot as plt

A = np.array([[3., 1., 0.], [1, 3., 1.], [0., 1., 3.]])
b = np.array([1., -1., 2.])
C = np.array([[1., 2., 0], [2., 0. ,0], [-1., 2., 0]])
f = np.array([1., 0, 2])

x_qp = solve_qp(A, -b, C, f)
print(f"QP solution: {x_qp=}")

QP solution: x_qp=array([ 0.   , -0.625,  0.875])


## Implementing gradient descent method ourselves
To solve the above problem we need to solve iteratively
$$ A u  - b+ C^\intercal \lambda = 0, \quad Cu \leq f, \quad \lambda \geq 0, \quad \lambda_i [Cu-f]_i = 0$$
To solve the linear system we consider the gradient descent method called Conjugate Gradient, and we use UZAWA algorithm to take care of the constrains

In [12]:
from numpy import linalg as la
def gc(A, b, x0, η, Imax):
    '''
    Solves the linear system Ax = b using Conjugate gradient
    '''
    x = x0
    r = b - A@x #residual
    d = r #initial descent direction
    n0 = np.dot(r,r)
    n1 = n0
    res = []
    i = 0
    while n0 > η and i < Imax:
        z = A@d
        α = n0 / (np.dot(z,d))
        x = x + α * d
        r = r - α * z
        n1 = np.dot(r,r)
        β = n1 / n0
        d = r + β * d
        n0 = n1
        i = i+1
        res = [res, np.sqrt(n1)]
        if np.sqrt(n1) > 1e10:
            break
    return x, i, res

def uzawa(A, b, C, f, λ0, ρ, η, Imax):
    '''
    Solves the linear system Ax  =  b - C^T*l using Conjugate gradient
    Then projects l using the constraints
    '''
    λ = λ0
    n0 = 1 + η
    res = []
    i = 0
    ηc = 1e-16
    Imaxc = 1000
    x = np.zeros(len(b))
    while n0 > η and i < Imax:
        [x, i, res] = gc(A, b - np.transpose(C)@λ, x, ηc, Imaxc)
        λ = np.maximum(np.zeros_like(λ), λ + ρ *( C@x -f))
        n0 = la.norm(np.maximum(np.zeros_like(λ), C@x-f))
        i = i+1
        res = [res, n0]
    return x, λ, i, res
        

In [13]:
A = np.array([[3., 1., 0.], [1, 3., 1.], [0., 1., 3.]])
b = np.array([1., -1., 2.])
C = np.array([[1., 2., 0], [2., 0. ,0], [-1., 2., 0]])
f = np.array([1., 0, 2])

λ0 = np.zeros(len(f))
ρ = 1e-5
η = 1e-16
Imax = 1000
[x, λ, i, res] = uzawa(A, b, C, f, λ0, ρ, η, Imax)

In [14]:
print(x)
print(λ)
print(C@x - f)
print(A@x-b + np.transpose(C)@λ)
print(f"Error between the 2 methods:", la.norm(x_qp - x))

[-3.83953019e-09 -6.25000000e-01  8.75000000e-01]
[0.         0.81250001 0.        ]
[-2.25000000e+00 -7.67906037e-09 -3.25000000e+00]
[-1.45324819e-09 -3.48754536e-09  1.16237642e-09]
Error between the 2 methods: 3.859501170436197e-09


In [15]:
%precision 3

'%.3f'

### Conjugate gradient: test and validation
We compare below the direct solver from Python and CG to solve Ax = b

In [16]:
solu = la.inv(A)@b
solu2 = gc(A, b, np.zeros_like(b), η, Imax)

In [17]:
print(solu)
print(solu2[0])

[ 0.619 -0.857  0.952]
[ 0.619 -0.857  0.952]
